In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time

In [124]:
def setup_driver():
    # Get the path for the ChromeDriver
    driver_path = ChromeDriverManager().install()

    # Set up Chrome options or capabilities (if needed)
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Create a Chrome service with the driver path
    chrome_service = Service(driver_path)

    # Initialize the Chrome WebDriver with options and service
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    return driver

def scrape_restaurant_links(driver, latitude, longitude, first_open):
    try:
        params = {
            "latitude": latitude,
            "longitude": longitude,
            "accuracy": 100
        }
        driver.execute_cdp_cmd("Emulation.setGeolocationOverride", params)

        driver.get("https://shop.ichefpos.com/explore")

        if first_open:
            # Wait until the dialog box is visible
            wait = WebDriverWait(driver, 30)
            dialog_box = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@role='dialog']")))

            # Find the "Share Location" button inside the dialog box
            share_location_button = dialog_box.find_element(By.XPATH, ".//button[contains(., '分享位置')]")
            share_location_button.click()
        
        # Wait for the button to be clickable
        wait = WebDriverWait(driver, 10)
        button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".TakeoutFinderCircleButton__CircleButton-kxzgzF")))

        # Click the button
        button.click()

        wait = WebDriverWait(driver, 10)
        # Wait for elements to load after clicking the button
        elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//*[contains(@id, 'restaurant-card')]")))

        shop_links = []
        shop_ids = []
        # Loop through each element to find anchor tags and extract href attribute
        for element in elements:
            anchor_tags = element.find_elements(By.TAG_NAME, 'a')
            for anchor_tag in anchor_tags:
                href = anchor_tag.get_attribute("href")
                id = anchor_tag.get_attribute("id")
                shop_links.append(href)
                shop_ids.append(id)
        print(len(shop_ids), len(shop_links))
        return shop_links, shop_ids

    except Exception as e:
        print(f'An error occurred: {str(e)}')
        return [], []

In [118]:
def store_restaurant_data(driver, latitude, longitude, first_open, filename='restaurant_data.json'):
    # Get the restaurant links and IDs
    restaurant_links, restaurant_ids = scrape_restaurant_links(driver, latitude, longitude, first_open)
    print(f'{len(restaurant_ids)} shops {len(restaurant_links)}')

    # Create a list of dictionaries to store the data
    restaurant_data = []
    for link, id in zip(restaurant_links, restaurant_ids):
        restaurant_data.append({'id': id, 'link': link})

    # Load existing data from the file (if any)
    try:
        with open(filename, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    # Append the new data to the existing data
    existing_data.extend(restaurant_data)

    # Write the updated data back to the file
    with open(filename, 'w') as file:
        json.dump(existing_data, file, indent=4)


In [135]:
# Taipei
north = 25.0500
west = 121.5000
east = 121.6000
south = 25.0100

step = 0.003

coords = []
current_lat = north
while current_lat >= south:
    current_lon = west
    while current_lon <= east:
        coords.append([current_lat, current_lon])
        current_lon += step
    current_lat -= step

In [136]:
len(coords)

476

In [137]:
driver = setup_driver()
first_open = True
for coord in coords:
    lat = coord[0]
    long = coord[1]
    store_restaurant_data(driver, lat, long, first_open)
    first_open = False
    time.sleep(3)
driver.quit()

8 8
8 shops 8
16 16
16 shops 16
40 40
40 shops 40
48 48
48 shops 48
66 66
66 shops 66
An error occurred: Message: 
Stacktrace:
0   chromedriver                        0x0000000102ea10f8 chromedriver + 4595960
1   chromedriver                        0x0000000102e98e63 chromedriver + 4562531
2   chromedriver                        0x0000000102a9c39a chromedriver + 381850
3   chromedriver                        0x0000000102ae5f08 chromedriver + 683784
4   chromedriver                        0x0000000102ae6191 chromedriver + 684433
5   chromedriver                        0x0000000102b2ab14 chromedriver + 965396
6   chromedriver                        0x0000000102b0816d chromedriver + 823661
7   chromedriver                        0x0000000102b2814d chromedriver + 954701
8   chromedriver                        0x0000000102b07ee3 chromedriver + 823011
9   chromedriver                        0x0000000102ad8be4 chromedriver + 629732
10  chromedriver                        0x0000000102ad979e ch

KeyboardInterrupt: 